In [1]:
%%bash
chia version
cdv --version
python --version

1.6.1b4.dev1
cdv, version 1.1.2
Python 3.10.6


In [2]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G1Element, G2Element)
from chia.types.blockchain_format.program import Program
from chia.wallet.puzzles import (p2_delegated_puzzle_or_hidden_puzzle)
# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../code")
from utils import (load_program, print_program, print_puzzle, print_json)
import singleton_utils
import tic_tac_toe

# puzzles
tic_tac_toe_puzzle = load_program("../code/tic-tac-toe.clsp", ["../code"])
coin_puzzle = load_program("../code/coin.clsp", ["../code"])
terminate_puzzle = load_program("../code/terminate-game.clsp", ["../code"])

waiting_room_puzzle = load_program("../code/waiting-room.clsp", ["../code"])

In [3]:
# from setup-sim notebook
# alice 4070016934
# bob 861183802
alice_pk = G1Element.from_bytes(bytes.fromhex("96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010"))
alice_puzzle_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice_pk).get_tree_hash()
bob_pk = G1Element.from_bytes(bytes.fromhex("b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a"))
bob_puzzle_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(bob_pk).get_tree_hash()

# prepare players info
player_one_info = Program.to([alice_pk, alice_puzzle_hash])
player_two_info = Program.to([bob_pk, bob_puzzle_hash])

In [4]:
%%bash
# two waiting-room-coins
cdv rpc coinrecords --by puzzlehash "3760683b29ed6bc3af3b19f32a3f3e13b5cb81cd9e7d586aea80e88b2e9ca4f4"
cdv rpc coinrecords --by puzzlehash "1d47e36dec5c853aff4d71fa4cfe2bd2c7b1d7de5c016dda3fa05cc5775aab6a"

[
    {
        "coin": {
            "amount": 1000000000001,
            "parent_coin_info": "0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b",
            "puzzle_hash": "0x3760683b29ed6bc3af3b19f32a3f3e13b5cb81cd9e7d586aea80e88b2e9ca4f4"
        },
        "coinbase": false,
        "confirmed_block_index": 6,
        "spent_block_index": 0,
        "timestamp": 1664993355
    }
]
[
    {
        "coin": {
            "amount": 1000000000000,
            "parent_coin_info": "0xfc669e48c63de848a7159610118f21d96efeb328af1876afd91a17cf1fecda97",
            "puzzle_hash": "0x1d47e36dec5c853aff4d71fa4cfe2bd2c7b1d7de5c016dda3fa05cc5775aab6a"
        },
        "coinbase": false,
        "confirmed_block_index": 6,
        "spent_block_index": 0,
        "timestamp": 1664993355
    }
]


# Create Game Coin
> Two waiting room coins available from Alice and Bob

[waiting-room.clsp](../code/waiting-room.clsp)
```clojure
(mod (OWNER_PUZZLE_HASH PK action)
```

In [5]:
from chia.util.bech32m import encode_puzzle_hash
## alice
alice_waiting_room_puzzle = waiting_room_puzzle.curry(
    alice_puzzle_hash,
    alice_pk
)
alice_waiting_room_puzzle_hash = alice_waiting_room_puzzle.get_tree_hash()
alice_waiting_room_address = encode_puzzle_hash(alice_waiting_room_puzzle_hash, "txch")
alice_waiting_room_address

'txch1xasxswefa44u8temr8ej50e7zw6uhqwdne74s6h2sr5gkt5u5n6qlyu2de'

In [6]:
## bob
bob_waiting_room_puzzle = waiting_room_puzzle.curry(
    bob_puzzle_hash,
    bob_pk
)
bob_waiting_room_puzzle_hash = bob_waiting_room_puzzle.get_tree_hash()
bob_waiting_room_address = encode_puzzle_hash(bob_waiting_room_puzzle_hash, "txch")
bob_waiting_room_address

'txch1r4r7xm0vtjzn4l6dw8ayel3t6trmr477tsqkmk3l5pwv2a664d4q6q6pg6'

## Terminate Puzzle (Curried)
[terminate-game.clsp](../code/terminate-game.clsp)
```clojure
(mod (IS_SINGLETON PLAYER_ONE_HASH PLAYER_TWO_HASH P2_AMOUNT play_result)
```

In [7]:
player_amount = 1_000_000_000_000
player_fee = 50_000_000
p2_amount = player_amount - player_fee
game_amount = (player_amount * 2) + 1 # odd amount for singleton
curried_terminate_puzzle = terminate_puzzle.curry(
    1, # Terminate Singleton Coin
    alice_puzzle_hash,
    bob_puzzle_hash,
    p2_amount
)

## Coin Puzzle (Curried)
[coin.clsp](../code/coin.clsp)
```clojure
(mod (MOD TERMINATE_PUZZLE PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE AMOUNT position)
```

In [8]:
def get_coin_puzzle(board, player):
    # (mod (BOARD V pos)
    curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
            Program.to(board), 
            Program.to(player)
        ) 

    #(mod (MOD PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE amount position)
    curried_coin_puzzle = coin_puzzle.curry(
        coin_puzzle,
        curried_terminate_puzzle,
        player_one_info,
        player_two_info,
        curried_tic_tac_toe_puzzle,
        game_amount)
    return curried_coin_puzzle

curried_coin_puzzle = get_coin_puzzle([' '] * 9, 'x')

## Launcher Coin

In [9]:
from chia.rpc.full_node_rpc_client import FullNodeRpcClient
from chia.util.config import load_config
from chia.util.default_root import DEFAULT_ROOT_PATH
from chia.util.ints import uint16

config = load_config(DEFAULT_ROOT_PATH, "config.yaml")
selected_network = "mainnet"
genesis_challenge = config["farmer"]["network_overrides"]["constants"][selected_network]["GENESIS_CHALLENGE"]

self_hostname = config["self_hostname"] # localhost
full_node_rpc_port = config["full_node"]["rpc_port"] # 8555
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(alice_waiting_room_puzzle_hash)
full_node_client.close()
await full_node_client.await_closed()

alice_waiting_room_coin = coin_records[0].coin
print(f'alice_waiting_room_coin:\n {alice_waiting_room_coin.to_json_dict()}')
print()
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(bob_waiting_room_puzzle_hash)
full_node_client.close()
await full_node_client.await_closed()

bob_waiting_room_coin = coin_records[0].coin
print(f'bob_waiting_room_coin:\n {bob_waiting_room_coin.to_json_dict()}')

alice_waiting_room_coin:
 {'parent_coin_info': '0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b', 'puzzle_hash': '0x3760683b29ed6bc3af3b19f32a3f3e13b5cb81cd9e7d586aea80e88b2e9ca4f4', 'amount': 1000000000001}

bob_waiting_room_coin:
 {'parent_coin_info': '0xfc669e48c63de848a7159610118f21d96efeb328af1876afd91a17cf1fecda97', 'puzzle_hash': '0x1d47e36dec5c853aff4d71fa4cfe2bd2c7b1d7de5c016dda3fa05cc5775aab6a', 'amount': 1000000000000}


In [10]:
from chia.types.blockchain_format.coin import Coin
from chia.types.coin_spend import CoinSpend
from chia.wallet.puzzles import singleton_top_layer_v1_1

# prepare launcher coin
launcher_coin = Coin(
    alice_waiting_room_coin.name(), # waiting_room's coin spend creates the launcher coin
    singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH, 
    game_amount
)

launcher_id = launcher_coin.name()
print(f'\nlauncher id: {launcher_id.hex()}')

singleton_struct = (
    singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
    (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
)

singleton_puzzle = singleton_top_layer_v1_1.SINGLETON_MOD.curry(
    singleton_struct,
    curried_coin_puzzle, # tic tac toe coin puzzle
)
launcher_coin


launcher id: dce355645e65a0dc159b9695aacde980bd1a00aacabfdb73179f1d4649f68520


Coin { parent_coin_info: 190a99274f2623de86ce8d243914ca492069318fda0144c4daa0ece2b3ccd8c7, puzzle_hash: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9, amount: 2000000000001 }

### Singleton Launcher
[singleton_top_layer.clvm](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/singleton_top_layer.clvm)
[singleton_launcher.clvm](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/singleton_launcher.clvm)
```clojure
(mod (singleton_full_puzzle_hash amount key_value_list)
```

In [13]:
launcher_solution = Program.to(
    [
        singleton_puzzle.get_tree_hash(),
        game_amount,
        [
            ("game", "tic tac toe"), 
            ("player one", alice_pk),
            ("player two", bob_pk),
        ]
    ]
)
print_program(launcher_solution)

launcher_announcement = launcher_solution.get_tree_hash()
print(f'\nlauncher_announcement:\n{launcher_announcement}')

launcher_coin_spend = CoinSpend(
    launcher_coin,
    singleton_top_layer_v1_1.SINGLETON_LAUNCHER,
    launcher_solution
)
print_json(launcher_coin_spend.to_json_dict())

(0xed888b7e0062ad1ec762af9f4c516b3bf753fd715183dc16e0e8d95e5b35026c 0x01d1a94a2001 (("game" . "tic tac toe") ("player one" . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) ("player two" . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a)))

launcher_announcement:
c1e2fd0d53a75ba89e155f2a61a690a6b2019212d30e883238b7eafd96b52371
{
    "coin": {
        "amount": 2000000000001,
        "parent_coin_info": "0x190a99274f2623de86ce8d243914ca492069318fda0144c4daa0ece2b3ccd8c7",
        "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9"
    },
    "puzzle_reveal": "0xff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0d

In [14]:
%%bash
opd ff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080
opd ffa0ed888b7e0062ad1ec762af9f4c516b3bf753fd715183dc16e0e8d95e5b35026cff8601d1a94a2001ffffff8467616d658b7469632074616320746f65ffff8a706c61796572206f6e65b096f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010ffff8a706c617965722074776fb0b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a8080

(a (q 4 (c 4 (c 5 (c 11 ()))) (c (c 10 (c (a 14 (c 2 (c (c 5 (c 11 (c 23 ()))) ()))) ())) ())) (c (q 51 60 2 (i (l 5) (q 11 (q . 2) (a 14 (c 2 (c 9 ()))) (a 14 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1))
(0xed888b7e0062ad1ec762af9f4c516b3bf753fd715183dc16e0e8d95e5b35026c 0x01d1a94a2001 (("game" . "tic tac toe") ("player one" . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) ("player two" . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a)))


In [15]:
%%bash
brun '(a (q 4 (c 4 (c 5 (c 11 ()))) (c (c 10 (c (a 14 (c 2 (c (c 5 (c 11 (c 23 ()))) ()))) ())) ())) (c (q 51 60 2 (i (l 5) (q 11 (q . 2) (a 14 (c 2 (c 9 ()))) (a 14 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1))' '(0xed888b7e0062ad1ec762af9f4c516b3bf753fd715183dc16e0e8d95e5b35026c 0x01d1a94a2001 (("game" . "tic tac toe") ("player one" . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) ("player two" . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a)))'

((51 0xed888b7e0062ad1ec762af9f4c516b3bf753fd715183dc16e0e8d95e5b35026c 0x01d1a94a2001) (60 0xc1e2fd0d53a75ba89e155f2a61a690a6b2019212d30e883238b7eafd96b52371))


In [16]:
print(f'launcher_coin_announcement:\n{launcher_announcement}')
print(f'\nlauncher_coin_id:\n{launcher_id.hex()}')

# announcementID == sha256(coinID + message).
from chia.util.hash import std_hash
launcher_coin_announcement = std_hash(launcher_id + launcher_announcement)
print(f'\nlauncher_coin_announcement:\n{launcher_coin_announcement}')

launcher_coin_announcement:
c1e2fd0d53a75ba89e155f2a61a690a6b2019212d30e883238b7eafd96b52371

launcher_coin_id:
dce355645e65a0dc159b9695aacde980bd1a00aacabfdb73179f1d4649f68520

launcher_coin_announcement:
fcef5fea1795408cfe077f06ddaaeb056925e34425e9ee67ab231addf8b8672c


In [17]:
alice_waiting_room_coin_spend = CoinSpend(
    alice_waiting_room_coin,
    alice_waiting_room_puzzle,
    Program.to([
        launcher_coin_announcement,
        1 # CREATE-GAME
    ])
)
print_json(alice_waiting_room_coin_spend.to_json_dict())

{
    "coin": {
        "amount": 1000000000001,
        "parent_coin_info": "0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b",
        "puzzle_hash": "0x3760683b29ed6bc3af3b19f32a3f3e13b5cb81cd9e7d586aea80e88b2e9ca4f4"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ffff09ff12ff2f80ffff01ff04ffff04ff1cffff01ff648080ffff04ffff04ff1affff04ff05ffff01ff8600e8d4a51000808080ffff04ffff04ff10ffff04ff0bffff04ff18ff80808080ff80808080ffff01ff04ffff04ff14ffff04ff17ff808080ffff02ffff03ffff09ff16ff2f80ffff01ff04ffff04ff1affff04ff1effff01ff8601d1a94a2001808080ff8080ff8080ff01808080ff0180ffff04ffff01ffffff329072657475726e20746f2073656e646572ff3d52ffff81ff33ff01a0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ff018080ffff04ffff01a0d210b94e1c190321d74e68fa697a8da3dce5846bbaf9f2783ba345c431123387ffff04ffff01b096f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010ff01808080",
    "solution": "0xffa0fcef5fea1795408c

In [18]:
%%bash
opd ff02ffff01ff02ffff01ff02ffff03ffff09ff12ff2f80ffff01ff04ffff04ff1cffff01ff648080ffff04ffff04ff1affff04ff05ffff01ff8600e8d4a51000808080ffff04ffff04ff10ffff04ff0bffff04ff18ff80808080ff80808080ffff01ff04ffff04ff14ffff04ff17ff808080ffff02ffff03ffff09ff16ff2f80ffff01ff04ffff04ff1affff04ff1effff01ff8601d1a94a2001808080ff8080ff8080ff01808080ff0180ffff04ffff01ffffff329072657475726e20746f2073656e646572ff3d52ffff81ff33ff01a0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ff018080ffff04ffff01a0d210b94e1c190321d74e68fa697a8da3dce5846bbaf9f2783ba345c431123387ffff04ffff01b096f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010ff01808080
opd ffa0fcef5fea1795408cfe077f06ddaaeb056925e34425e9ee67ab231addf8b8672cff0180

(a (q 2 (q 2 (i (= 18 47) (q 4 (c 28 (q 100)) (c (c 26 (c 5 (q 0x00e8d4a51000))) (c (c 16 (c 11 (c 24 ()))) ()))) (q 4 (c 20 (c 23 ())) (a (i (= 22 47) (q 4 (c 26 (c 30 (q 0x01d1a94a2001))) ()) ()) 1))) 1) (c (q ((50 . "return to sender") 61 . 82) (-1 . 51) 1 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9) 1)) (c (q . 0xd210b94e1c190321d74e68fa697a8da3dce5846bbaf9f2783ba345c431123387) (c (q . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) 1)))
(0xfcef5fea1795408cfe077f06ddaaeb056925e34425e9ee67ab231addf8b8672c 1)


In [19]:
%%bash
brun '(a (q 2 (q 2 (i (= 18 47) (q 4 (c 28 (q 100)) (c (c 26 (c 5 (q 0x00e8d4a51000))) (c (c 16 (c 11 (c 24 ()))) ()))) (q 4 (c 20 (c 23 ())) (a (i (= 22 47) (q 4 (c 26 (c 30 (q 0x01d1a94a2001))) ()) ()) 1))) 1) (c (q ((50 . "return to sender") 61 . 82) (-1 . 51) 1 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9) 1)) (c (q . 0xd210b94e1c190321d74e68fa697a8da3dce5846bbaf9f2783ba345c431123387) (c (q . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) 1)))' '(0xfcef5fea1795408cfe077f06ddaaeb056925e34425e9ee67ab231addf8b8672c 1)'

((61 0xfcef5fea1795408cfe077f06ddaaeb056925e34425e9ee67ab231addf8b8672c) (51 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 0x01d1a94a2001))


In [20]:
bob_waiting_room_coin_spend = CoinSpend(
    bob_waiting_room_coin,
    bob_waiting_room_puzzle,
    Program.to([
        launcher_coin_announcement,
        0 # SPEND
    ])
)
print_json(bob_waiting_room_coin_spend.to_json_dict())

{
    "coin": {
        "amount": 1000000000000,
        "parent_coin_info": "0xfc669e48c63de848a7159610118f21d96efeb328af1876afd91a17cf1fecda97",
        "puzzle_hash": "0x1d47e36dec5c853aff4d71fa4cfe2bd2c7b1d7de5c016dda3fa05cc5775aab6a"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ffff09ff12ff2f80ffff01ff04ffff04ff1cffff01ff648080ffff04ffff04ff1affff04ff05ffff01ff8600e8d4a51000808080ffff04ffff04ff10ffff04ff0bffff04ff18ff80808080ff80808080ffff01ff04ffff04ff14ffff04ff17ff808080ffff02ffff03ffff09ff16ff2f80ffff01ff04ffff04ff1affff04ff1effff01ff8601d1a94a2001808080ff8080ff8080ff01808080ff0180ffff04ffff01ffffff329072657475726e20746f2073656e646572ff3d52ffff81ff33ff01a0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ff018080ffff04ffff01a03ac42b719d0b34879714d1e601721f08ba317862d2118fa16aa7e8bf1a19657bffff04ffff01b0b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64aff01808080",
    "solution": "0xffa0fcef5fea1795408c

In [21]:
%%bash
opd ff02ffff01ff02ffff01ff02ffff03ffff09ff12ff2f80ffff01ff04ffff04ff1cffff01ff648080ffff04ffff04ff1affff04ff05ffff01ff8600e8d4a51000808080ffff04ffff04ff10ffff04ff0bffff04ff18ff80808080ff80808080ffff01ff04ffff04ff14ffff04ff17ff808080ffff02ffff03ffff09ff16ff2f80ffff01ff04ffff04ff1affff04ff1effff01ff8601d1a94a2001808080ff8080ff8080ff01808080ff0180ffff04ffff01ffffff329072657475726e20746f2073656e646572ff3d52ffff81ff33ff01a0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ff018080ffff04ffff01a03ac42b719d0b34879714d1e601721f08ba317862d2118fa16aa7e8bf1a19657bffff04ffff01b0b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64aff01808080
opd ffa0fcef5fea1795408cfe077f06ddaaeb056925e34425e9ee67ab231addf8b8672cff8080

(a (q 2 (q 2 (i (= 18 47) (q 4 (c 28 (q 100)) (c (c 26 (c 5 (q 0x00e8d4a51000))) (c (c 16 (c 11 (c 24 ()))) ()))) (q 4 (c 20 (c 23 ())) (a (i (= 22 47) (q 4 (c 26 (c 30 (q 0x01d1a94a2001))) ()) ()) 1))) 1) (c (q ((50 . "return to sender") 61 . 82) (-1 . 51) 1 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9) 1)) (c (q . 0x3ac42b719d0b34879714d1e601721f08ba317862d2118fa16aa7e8bf1a19657b) (c (q . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a) 1)))
(0xfcef5fea1795408cfe077f06ddaaeb056925e34425e9ee67ab231addf8b8672c ())


In [22]:
%%bash
brun '(a (q 2 (q 2 (i (= 18 47) (q 4 (c 28 (q 100)) (c (c 26 (c 5 (q 0x00e8d4a51000))) (c (c 16 (c 11 (c 24 ()))) ()))) (q 4 (c 20 (c 23 ())) (a (i (= 22 47) (q 4 (c 26 (c 30 (q 0x01d1a94a2001))) ()) ()) 1))) 1) (c (q ((50 . "return to sender") 61 . 82) (-1 . 51) 1 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9) 1)) (c (q . 0x3ac42b719d0b34879714d1e601721f08ba317862d2118fa16aa7e8bf1a19657b) (c (q . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a) 1)))' '(0xfcef5fea1795408cfe077f06ddaaeb056925e34425e9ee67ab231addf8b8672c ())'

((61 0xfcef5fea1795408cfe077f06ddaaeb056925e34425e9ee67ab231addf8b8672c))


In [23]:
from chia.types.spend_bundle import SpendBundle
spend_bundle = SpendBundle([
        launcher_coin_spend,
        alice_waiting_room_coin_spend,
        bob_waiting_room_coin_spend
    ], 
    G2Element() # empty signature
)
print_json(spend_bundle.to_json_dict(include_legacy_keys = False, exclude_modern_keys = False))

{
    "aggregated_signature": "0xc00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
    "coin_spends": [
        {
            "coin": {
                "amount": 2000000000001,
                "parent_coin_info": "0x190a99274f2623de86ce8d243914ca492069318fda0144c4daa0ece2b3ccd8c7",
                "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9"
            },
            "puzzle_reveal": "0xff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080",
            "solution": "0xffa0ed888b7e0062ad1ec762af9f4c516b3bf753fd715